# Tanzania Water Well Classification Project: Does it Need Repair?

## **About**

#### The purpose of this notebook is to share a summary of the methodology used during this project. Here you can find the goals of the project, what factors were taken into account during the Exploratory Data Analysis(EDA) phase, examples from the model building process, our final model and evaluation, and considerations and decisions taken along the way. 

### ***Poject Goals***

#### The goal of this project is to build a classification model to know if a waterpump is in need of repair. 

#### The problem this project is addressing is access to water by way of community waterpoints. Access to water is an important issue that has reverberations in the social and economic aspects of a society. The [World Health Organization writes](https://www.who.int/news-room/fact-sheets/detail/drinking-water#:~:text=Safe%20and%20readily%20available%20water,contribute%20greatly%20to%20poverty%20reduction.), “Safe and readily available water is important for public health, whether it is used for drinking, domestic use, food production or recreational purposes. Improved water supply and sanitation, and better management of water resources, can boost countries’ economic growth and can contribute greatly to poverty reduction.” 

#### The water pumps we are looking at in our modeling are meant to provide potable water. If these water pumps fail, that community’s availability of drinking water is impacted. Reduced availability of working water pumps means an increased use of the functional ones, which could mean a reduction in that water pumps lifespan before it needs repairs. Being able to know which waterpoints need maintenance can help that community have a minimal interruption of service.

#### This model will be used by the Tanzanian Ministry of Water to assess which water pumps need to be repaired. This model helps the goals of the Tanzanian Ministry of Water to provide access to water to its citizens. By knowing which water pumps need repairs, the Ministry of Water can implement better maintenance strategies. Moreover, this model and implementation of outcomes can help towards [The Tanzanian Development Vision 2025](https://mof.go.tz/mofdocs/overarch/vision2025.htm) to have universal access to safe water by the year 2025.

### ***The Data***

#### The **data comes from** [Taarifa](http://taarifa.org/) who sources it from the [Tanzanian Ministry of Water](https://www.maji.go.tz/). The datasets were downloaded from [Driven Data’s “Pump it Up: Data Mining the Water Table”](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/) competition.

#### Our target is to classify the water pumps into one of three possible categories:
1. `functional` - the waterpoint is operational and there are no repairs needed
2. `functional needs repair` - the waterpoint is operational, but needs repairs
3. `non functional` - the waterpoint is not operational


## **Exploratory Data Analysis (EDA)**

#### A few of the questions we explored before going into our modeling were:
    - What features are available to us? Do we need all of them?
    - What format are our features in? 
    - Are there missing values in our datasets? how will we account for those?

PAUL'S CODE AND COMMENTS
ADD CODE HERE
-column names...some examples of the on categorical, looked for reduntant variables in the catergorical data
- dtypes-- write out thinking 
- checked for missing values, discuss how will take care of them when building model


## **First Simple Model**

In [1]:
#Import custom functions

import sys
import pathlib
src_path = pathlib.Path().absolute()/"src"
sys.path.append(str(src_path))
import data_functions


from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix

import pandas as pd
import numpy as np


## **Data Cleaning**

Before we can begin modeling we first have to clean the data by dropping any features that aren't relevant to the modeling process as well as scalling numeric features and one hot encoding categorical features

In [24]:
# Import data to DataFrames
data_path = pathlib.Path().absolute()/"data"
x_train, x_test, y_train = data_functions.get_dataframes(data_path)

# Perform data preprocessing to improve modeling results
x_train_pp, y_train_pp = data_functions.data_preprocessing(x_train, y_train)

# Perform a train test split for validation after modeling
x_tr_split, x_te_split, y_tr_split, y_te_split = train_test_split(x_train_pp, y_train_pp, random_state=2020)

## **Initial Model Testing**

PAUL adds code for the confustion matrix with the 3 classifications. KNN and RF?
ADD CODE for example models with the 3 classifications

### ***Random Forest Classifier***

### ***KNeighbor Classifier***

In [ ]:
kn3_2 = KNeighborsClassifier(n_neighbors = 15)
kn3_2.fit(X_tr_sm,y_tr_sm)
y_pred_kn3_2=kn3_2.predict(x_val_test)
print(classification_report(y_val_test, y_pred_kn3_2))
plot_confusion_matrix(kn3_2, x_val_test,y_val_test)

In [ ]:
#example of hypertuning, did not improve recall, only accuracy as may be expected
kn3_5 = KNeighborsClassifier(n_neighbors = 3, weights = 'distance', p = 1, algorithm = 'brute')
kn3_5.fit(X_tr_sm,y_tr_sm)
y_pred_kn3_5=kn3_5.predict(x_val_test)
print(classification_report(y_val_test, y_pred_kn3_5))
plot_confusion_matrix(kn3_5, x_val_test,y_val_test)

## **Model Iteration**

After tesing various models and up-sampling techniques such as SMOTE, we were unable to reliably classify our minority class 'functional, needs repair'. Because of this we decided that the most important thing is to classify any well that needed repair and that combining 'functional, needs repair' and 'non-functional' would be an effective way to eliminate our minority class while still answering the fundamental question of which wells need repair and which wells are completely functional.

In [25]:
# creating a function that converts the 'functional' class to 1 and every other class to 0
bin_y = lambda x: 1 if x == 'functional' else 0
# apply the function to y_train and y_test
y_tr_split = pd.DataFrame(y_tr_split['status_group'].apply(bin_y), index = y_tr_split.index, columns = y_tr_split.columns)
y_te_split = pd.DataFrame(y_te_split['status_group'].apply(bin_y), index = y_te_split.index, columns = y_te_split.columns)

### ***Random Forest Classifier***

PAUL Add 1 or two examples?

### ***KNeighbor Classifier***

In [ ]:
#Example 1. This was model iteration number 2 begining to use hyperparameters to tune
kn_2 = KNeighborsClassifier(n_neighbors = 15)
kn_2.fit(x_val,y_tr_final)
y_pred_kn_2=kn_2.predict(x_val_test)
print(classification_report(y_te_final, y_pred_kn_2))

In [ ]:
#Example 2. This was model iteration number 8 using hyperparameter tuning
kn_8 = KNeighborsClassifier(n_neighbors = 2, weights = 'distance', p = 1, algorithm = 'brute')
kn_8.fit(x_val,y_tr_final)
y_pred_kn_8=kn_8.predict(x_val_test)
print(classification_report(y_te_final, y_pred_kn_8))

### ***Logistic Classification***

#### Hypertuned for regularization strength (C) and class_weight. First model turned out to be the best for this type of classification.

In [ ]:
lr_1= LogisticRegression(max_iter= 2000)
lr_1.fit(x_val,y_tr_final)
y_pred_lr_1=lr_1.predict(x_val_test)
print(classification_report(y_te_final, y_pred_lr_1))
plot_confusion_matrix(lr_1, x_val_test,y_te_final)

## **Final Model**

We came to our final model by grid searching with a random forest classifier. The best parameters found in the grid search were: max_features = 75, min_samples_leaf = 3, n_estimators = 200 

In [27]:
# creating a RandomForestClassifier object with our best parameters
final_model = RandomForestClassifier(max_features = 75, min_samples_leaf = 3, n_estimators = 200)

In [29]:
# Fitting our classifier on the training split
final_model.fit(x_tr_split, y_tr_split['status_group'])

RandomForestClassifier(max_features=75, min_samples_leaf=3, n_estimators=200)

In [30]:
# Scoring our model's accuracy on the testing split
final_model.score(x_te_split, y_te_split)

0.8098989898989899

## **Conclusion**